## Schema 1: Intensive care unit stay

![Cube 1](img/Cube1.png)

In [1]:
require 'sequel'

DB_MIMIC = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['mimiciii']
                          )

DB_OLAP = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['olap']
                          )

load '../etl/model.rb'
nil

#### Time dimension table

In [2]:
# Creation
DB_OLAP.drop_table?(:icu_stay_time)
DB_OLAP.create_table(:icu_stay_time) do
  DateTime :intime, primary_key: true
  String :day
  String :month
  String :year
end

# Population
Icustay.all.uniq.each do |icustay|
  time = icustay.intime
  year = time.year
  month = time.month
  day = time.day
  DB_OLAP[:icu_stay_time].insert(
    intime: time,
    day: "#{year}-#{month}-#{day}",
    month: "#{year}-#{month}",
    year: year.to_s
  )
end

nil

#### Patient age dimension table

In [12]:
def age_in_completed_years (bd, d)
    # Difference in years, less one if you have not had a birthday this year.
    a = d.year - bd.year
    a = a - 1 if (
         bd.month >  d.month or 
        (bd.month >= d.month and bd.day > d.day)
    )
    a
end

:age_in_completed_years

In [76]:
# Creation
DB_OLAP.drop_table?(:icu_stay_pat_age)
DB_OLAP.create_table(:icu_stay_pat_age) do
  Integer :age_id, primary_key: true
  String :age
  String :decade
end

# Population
Icustay.all.uniq.each do |icustay|
  patient = icustay.patient
  age = age_in_completed_years(patient.dob, icustay.intime)
  decade = age / 10
  decade_str = "#{decade}0 - #{decade}9"  
  if DB_OLAP[:icu_stay_pat_age].where(age_id: age).empty?
    DB_OLAP[:icu_stay_pat_age].insert(
      age_id: age,
      age: age.to_s,
      decade: decade_str
    )
  end
end

nil

#### Diagnose dimension table

In [2]:
def find_cat (icd9)
  if icd9.match /^(V|E)[0-9]*$/
    return "E and V codes: external causes of injury and supplemental classification"
  end
  icd9 = icd9[0..2].to_i
  if icd9 < 140
    return "001-139: infectious and parasitic diseases"
  end
  if icd9 < 240
    return "140-239: neoplasms"
  end
  if icd9 < 280
    return "240-279: endocrine, nutritional and metabolic diseases, and immunity disorders"
  end
  if icd9 < 290
    return "280-289: diseases of the blood and blood-forming organs"
  end
  if icd9 < 320
    return "290-319: mental disorders"
  end
  if icd9 < 390
    return "320-389: diseases of the nervous system and sense organs"
  end
  if icd9 < 460
    return "390-459: diseases of the circulatory system"
  end
  if icd9 < 520
    return "460-519: diseases of the respiratory system"
  end
  if icd9 < 580
    return "520-579: diseases of the digestive system"
  end
  if icd9 < 630
    return "580-629: diseases of the genitourinary system"
  end
  if icd9 < 680
    return "630-679: complications of pregnancy, childbirth, and the puerperium"
  end
  if icd9 < 710
    return "680-709: diseases of the skin and subcutaneous tissue"
  end
  if icd9 < 740
    return "710-739: diseases of the musculoskeletal system and connective tissue"
  end
  if icd9 < 760
    return "740-759: congenital anomalies"
  end
  if icd9 < 780
    return "760-779: certain conditions originating in the perinatal period"
  end
  if icd9 < 800
    return "780-799: symptoms, signs, and ill-defined conditions"
  end
  if icd9 < 1000
    return "800-999: injury and poisoning"
  end
  return ""
end 
    

:find_cat

In [4]:
#Creation
# Creation
DB_OLAP.drop_table?(:icu_stay_diagnose)
DB_OLAP.create_table(:icu_stay_diagnose) do
  String :icd9_code, primary_key: true
  String :icd9_cat
end

# Population
Icustay.all.uniq.each do |icustay|
  admission = icustay.admission
  admission.diagnoses_icd.each do |diagnose|
    icd9 = diagnose.icd9_code
    if DB_OLAP[:icu_stay_diagnose].where(icd9_code: icd9).empty?
      DB_OLAP[:icu_stay_diagnose].insert(
        icd9_code: icd9,
        icd9_cat: find_cat(icd9)
      )
    end
  end
end

nil

#### Fact table

In [ ]:
# Creation
DB_OLAP.drop_table?(:icustays)
DB_OLAP.create_table(:icustays) do
  Integer :id, primary_key: true
  # Measures
  DateTime :los
  Integer :death
  # Dimension keys 
  DateTime :intime
  String :icd9_code
  Integer :age_id
end

# Population
Icustay.all.uniq.each do |icustay|
  DB_OLAP[:admissions].insert(
    id: icustay.row_id,
    
    los: DateTime.strptime((icustay.outtime - icustay.intime).to_s, '%s')
    death: icustay.admission.hospital_expire_flag
    
    intime: icustay.intime,
    icd9_code: 
    age_id: age_in_completed_years(icustay.patient.dob, icustay.intime)
  )
end

nil

## XML implementation

## TESTS

In [48]:
DB_OLAP[:icu_stay_pat_age].where(age_id: 5).empty?

true

In [52]:
[1, 2, 3].methods.grep(/\?/)

[:include?, :any?, :empty?, :eql?, :frozen?, :all?, :one?, :none?, :member?, :instance_variable_defined?, :instance_of?, :kind_of?, :is_a?, :respond_to?, :nil?, :tainted?, :untrusted?, :equal?]

In [77]:
a = Icustay.first.intime
b = Icustay.first.patient.dob
age_in_completed_years(b, a)

70

In [3]:
a = "453"
if a =~ /^(V|E)[0-9]*/
  puts 'yes'
else
  puts 'no'
end

no


In [26]:
icustay = Icustay.first
DateTime.strptime((icustay.outtime - icustay.intime).to_s, '%s')
#Icustay.each do |icustay| 
#  puts "#{icustay.row_id}: #{icustay.admission.diagnosis}"
#end
Icustay[12763]
DiagnoseIcd.where(:hadm_id => Icustay[12763].hadm_id).all

[#<DiagnoseIcd @values={:row_id=>112513, :subject_id=>10027, :hadm_id=>199395, :seq_num=>1, :icd9_code=>"4280"}>, #<DiagnoseIcd @values={:row_id=>112514, :subject_id=>10027, :hadm_id=>199395, :seq_num=>2, :icd9_code=>"4240"}>, #<DiagnoseIcd @values={:row_id=>112515, :subject_id=>10027, :hadm_id=>199395, :seq_num=>3, :icd9_code=>"3970"}>, #<DiagnoseIcd @values={:row_id=>112516, :subject_id=>10027, :hadm_id=>199395, :seq_num=>4, :icd9_code=>"4266"}>, #<DiagnoseIcd @values={:row_id=>112517, :subject_id=>10027, :hadm_id=>199395, :seq_num=>5, :icd9_code=>"42731"}>, #<DiagnoseIcd @values={:row_id=>112518, :subject_id=>10027, :hadm_id=>199395, :seq_num=>6, :icd9_code=>"5859"}>, #<DiagnoseIcd @values={:row_id=>112519, :subject_id=>10027, :hadm_id=>199395, :seq_num=>7, :icd9_code=>"41401"}>, #<DiagnoseIcd @values={:row_id=>112520, :subject_id=>10027, :hadm_id=>199395, :seq_num=>8, :icd9_code=>"4019"}>, #<DiagnoseIcd @values={:row_id=>112521, :subject_id=>10027, :hadm_id=>199395, :seq_num=>9, :i